# Transaction Cost analysis

for different kernels

# Libraries and Data

In [71]:
import numpy as np
import pandas as pd
import sys
import os
import pickle
import cvxpy as cp
from scipy.optimize import minimize
from project_lib.backtest import *
from project_lib.utils import *
from project_lib.performance import *
from project_lib.analysis import *
from project_lib.portfolio import Portfolio

HOME_DIRECTORY = 'C:/Users/Harol/OneDrive/Documents/master computational finance/thesis/thesis_UCL/Code/Transaction Costs'
sys.path.append(HOME_DIRECTORY)

In [2]:
# import returns
with open(HOME_DIRECTORY + '/data/processed_daily_data/ret_subset.pkl', 'rb') as f:
    ret = pickle.load(f)

In [3]:
universe_size = 100
ret = ret.iloc[:1000, :universe_size]  # subset the data
ret = ret.iloc[(4):] # burn


In [67]:
prices = (1 + ret).cumprod()
prices = prices.iloc[:,:universe_size]

In [56]:
# import weights and correlations
gauss_w = pd.read_csv("gaussian_weights.csv")
tri_w = pd.read_csv("triangular_weights.csv")
epan_w = pd.read_csv("epanechnikov_weights.csv")
cca_w = pd.read_csv("sample_cca_weights.csv") # linear version

gauss_c = pd.read_csv("gaussian_correlations.csv")
tri_c = pd.read_csv("triangular_correlations.csv")
epan_c = pd.read_csv("epanechnikov_correlations.csv")

dfs = [gauss_w, tri_w, epan_w, gauss_c, tri_c, epan_c]

In [57]:
for df in dfs:
    df.set_index("date", inplace=True)
    df.index = pd.to_datetime(df.index)

In [58]:
cca_w.set_index("Unnamed: 0", inplace=True)
cca_w.index = pd.to_datetime(cca_w.index)

In [61]:
kernels = ["gaussian","triangular","epanechnikov"]

## preprocessing

In [62]:
def cleaning(x):
    a = [ele for ele in x.strip("[]").split(" ") if ele.strip()]
    a = [elem.replace("\n","") for elem in a]
    return np.asarray(a,dtype=float)

In [63]:
gauss_w  = gauss_w.applymap(cleaning)
tri_w = tri_w.applymap(cleaning)
epan_w = epan_w.applymap(cleaning)

## correcting a mistake regarding the correlations

In [ ]:
K_pl = np.linalg.inv(cov_R_half) @ (results.T @ results) @ np.linalg.inv(cov_R_half)
        
        # make sure it's sorted
        eigen_val, eigen_vec = np.linalg.eig(K_pl)
        order = np.argsort(eigen_val)[::-1]
        idx = np.empty_like(order)
        idx[order] = np.arange(len(order))
        eigen_vec[:] = eigen_vec[:, idx] 

In [101]:
# step 1: all column elements as one matrix
# step 2: eigenvalues
# step 3: eigenvalues in place
#for kernel in kernels:
for kernel in kernels:
    print(kernel)
    cp = all_weights[kernel].copy()
    for row in range(cp.shape[0]):
        cov_R_half = get_cov(ret.iloc[row:(row+250),:], method="sample", square_root=True)
        # re-extract the weights
        w = cov_R_half.T @ np.array(gauss_w.iloc[0,:].to_list()).T
        
        K = w @ w.T
        
        eigenval, eigenvec = np.linalg.eig(K)
        
        for col in range(all_corr[kernel].shape[1]):
            all_corr[kernel].iloc[row,col] = eigenval[col]


gaussian
triangular
epanechnikov


# No Transaction Costs Performance

# Transaction cost based on portfolios

Inspired from analytical solutions of optimal portfolio rebalancing, Ding Liu, 2019

In [34]:
all_weights = {'gaussian':gauss_w, 'triangular':tri_w,'epanechnikov':epan_w}
all_corr = {'gaussian':gauss_c, 'triangular':tri_c,'epanechnikov':epan_c}

In [206]:
def update_weights(new_weights,old_weights, corr, tcost, risk_aversion, pf_variance):
    """
        rebalance the weights depending on rebalancing costs, risk aversion, and portfolio variance
        
        inputs:
                new_weights   : matrix of all weights calculated using CCA at t [m x m]
                old_weights   : total portfolio weights at t-1                  [1 x m]
                corr          : correlations corresponding to new_weights at t  [1 x m]
                tcost         : transaction cost parameter                      [1 x 1]
                risk_aversion : risk aversion parameter                         [1 x 1]
                pf_variance   : current portfolio variance at t-1               [1 x 1]
        outputs:
                change_w : change in weights (compared to old weights) [1 x m]
    """
    # step 1 : multiply each portfolio by its correlation
    w = new_weights*np.array(corr)
    # step 2 : calculate total weights
    total_w = w.sum()
    # step 3 : calculate weights change
    change_w = old_weights - total_w
    # step 4 : initial transaction cost
    rebalance_cost = tcost * np.sum(np.abs(change_w))
    # step 5 : calculate which portfolios to include iteratively
    trade = False
    while(trade==False):
        # check whether it is worth rebalancing given total current turnover
        if np.sum(np.abs(change_w)) - (1/risk_aversion) * rebalance_cost / pf_variance < 0:
            # remove last canonical portfolio
            # unless we decide not to trade any portfolio -> end loop
            if w.shape[1]==0:
                trade = True
            else:
                w = np.delete(w, -1, 1)
                total_w = w.sum()
                change_w = old_weights - total_w
                rebalance_cost = tcost * np.sum(np.abs(change_w))
        else:
            trade = True
    # return the change in weights
    return change_w

In [ ]:
# function to iteratively calculate portfolio weights & profitability
def backtest_cca():
    return 0

# Transaction cost on asset level

Implementation of "Multiperiod portfolio optimization with multiple risky assets and general transaction costs", Mei, Demiguel, Nogales, 2016

In [41]:
def rebalancing(X,X_prev, rho, gamma, kappa, mu,sigma, lag, target="Markowitz"):
    """
        Function to calculate optimal rebalancing on asset level with proportional transaction costs.
        
        Implementation equation (2) in Multiperiod portfolio optimization with multiple risky assets
        and general transaction costs.
        
        Inputs:
                X      : target weights                   [1 x m]
                X_prev : previous weights                 [1 x m]
                rho    : discount rate                    [1 x 1]
                gamma  : absolute risk-aversion parameter [1 x 1]
                kappa  : transaction cost parameter       [1 x 1]
                mu     : mean returns                     [1 x m]
                sigma  : covariance of returns            [m x m]
                lag    : rebalancing horizon              [1 x 1]
        Output:
                new_w : new weights [1 x m]
    """
    m = len(X)
    
    # initiliase variable
    w = cp.Variable(m)
    
    constraints = []
    # objective function
    if target=="Markowitz":
        obj = cp.Maximize((1-rho)**lag * (w * mu - gamma/2 * w * sigma * w) - kappa*cp.norm(w - X_prev, 1))
    elif target=="Target":
        ########### PERFORM WITH SCIPY INSTEAD
        #obj = cp.Minimize(cp.norm(X - w + kappa * cp.norm(w - X_prev, 1)))
        #obj = cp.Minimize(cp.sum(X-w))
        #constraints = [cp.norm(w)<=1]
        arguments = (X, kappa, X_prev)
        res = minimize(minimize_target, x0=X, args=arguments)
        new_w = res.x
        
    elif target == "Simple":
        obj  = cp.Maximize((1-rho)**lag * w - kappa * cp.norm(w - X_prev, 1))
    # constraints
    # None first
    
    # solve problem
    #prob = cp.Problem(obj, constraints)
    #prob.solve(verbose = False)
    
    # new weights
    #new_w = np.array(w.value)
    
    
    return new_w
def minimize_target(w, X, kappa, X_prev):
    return np.linalg.norm(X-w + kappa * np.linalg.norm(w - X_prev,1),1)

In [42]:
def constant_rebalancing(weights, rho, gamma, kappa, returns, lag, target):
    """
        function to perform continuous rebalancing taking into account transaction costs
    
    """
    # create some variables    
    means = returns.rolling(250).mean().iloc[250:,:]
    #covariances = returns.rolling(250).cov()
    covariances = 1
    new_weights = weights.copy()
    
    new_weights.iloc[0,:] = new_weights.iloc[0,:]# / np.linalg.norm(new_weights.iloc[0,:])
    
    # first very basic function
    for i in range(1,weights.shape[0]):
        target_w = np.array(weights.iloc[i,:])#/np.linalg.norm(weights.iloc[i,:]))
        prev_w = np.array(new_weights.iloc[i-1,:])
        
        temp = rebalancing(target_w, prev_w, rho=rho,
                              gamma=gamma, kappa=kappa, mu=means.iloc[i,:], sigma=covariances, lag=lag, target=target)
        
        for j in range(len(temp)):
            new_weights.iloc[i,j] = temp[j]
    
    return new_weights


In [64]:
w1 = cca_w.iloc[:30,:]

In [83]:
test = constant_rebalancing(w1, rho=0, gamma=10e-6, kappa=0.01, returns=ret, lag=1, target="Target")


In [84]:
test1 = {"sample":test}

In [85]:
# build P&L
cov_models=["sample"]
pnl_results = dict()
ptf_ret = dict()
for model in cov_models:
    portfolio =  Portfolio(prices=prices.loc[test1[model].index], position=test1[model], period=0)
    ptf_ret[model] = portfolio.profit.to_frame(name="Profit")
    pnl_results[model] = portfolio.nav().to_frame(name="NAV")

In [86]:
build_table2(cov_models, ptf_ret)

,AV,SD,IR,VaR,MDD,P2T,P2P,P2PL,Calmar,Stability,Omega,Sortino,TailRatio,CSR,Kurtosis
sample,799.59,171.69,4.66,0.8,-1.28,3.0,NaN,30.0,6.25,0.78,2.07,7.83,1.3,2.69,0.04


In [87]:
build_table3(cov_models, test1, ptf_ret)

,TO,GL,PL,IR_net,herf,pos
sample,23.53,25.69,0.33,3.03,0.09,66.8


In [88]:
plotting(pnl_results, cov_models)

## impact of different levels of transaction costs

In [93]:
# in case I want to parallelize
#%%writefile rebalancing.py
#import numpy as np
#import pandas as pd
## import constant_rebalancing function
#def reb(cca_w,ret, tcost):
#    return constant_rebalancing(cca_w, rho=0, gamma=10e-6, kappa=tcost, returns=ret, lag=1, target="Target")

In [ ]:
tcosts = [0.001, 0.0025, 0.005, 0.01, 0.05, 0.1]

for tcost in tcosts:
    print("on tcost {}".format(tcost))
    tcost_weights = constant_rebalancing(w1, rho=0, gamma=10e-6, kappa=tcost, returns=ret, lag=1, target="Target")
    tcost_weights.to_csv("cca_tcostweights_"+str(tcost)+".csv")

on tcost 0.001
on tcost 0.0025
on tcost 0.005
